In [67]:
import pandas as pd
import numpy as np
import urllib.request
import re

In [96]:
df = pd.read_csv('dataset4500.csv')

In [97]:
df.head()

,artist,name,url,yt_links,Genre0,Genre1,Genre2,Genre3,Genre4,Genre5,Genre6,Genre7
0,Mariah Carey,All I Want for Christmas Is You,https://www.last.fm/music/Mariah+Carey/_/All+I...,https://www.youtube.com/watch?v=yXQViqx6GMY,christmas,pop,xmas,mariah carey,pop,rnb,female vocalists,NaN
1,Olivia Rodrigo,good 4 u,https://www.last.fm/music/Olivia+Rodrigo/_/goo...,https://www.youtube.com/watch?v=gNi_6U5Pm_o,pop rock,rock,pop punk,sour,pop,pop rock,indie pop,NaN
2,Doja Cat,Kiss Me More (feat. SZA),https://www.last.fm/music/Doja+Cat/_/Kiss+Me+M...,https://www.youtube.com/watch?v=0EVVKs6DQLo,planet her,pop,rnb,pop rap,2021,pop,rnb,rap
3,Wham!,Last Christmas,https://www.last.fm/music/Wham%21/_/Last+Chris...,https://www.youtube.com/watch?v=E8gmARGvPlI,christmas,80s,pop,xmas,pop,80s,british,NaN
4,Lil Nas X,INDUSTRY BABY (feat. Jack Harlow),https://www.last.fm/music/Lil+Nas+X/_/INDUSTRY...,https://www.youtube.com/watch?v=UTHLKHL_whs,rap,hip-hop,trap,pop rap,2021,hip-hop,rap,country rap


In [98]:
df_w_genres = df[~df[df.columns[-8:]].isna().all(axis=1)]

-39 песен

In [99]:
missing_yt = df_w_genres[df_w_genres.yt_links == "None"]
missing_yt.head()

,artist,name,url,yt_links,Genre0,Genre1,Genre2,Genre3,Genre4,Genre5,Genre6,Genre7
874,Offset,Ric Flair Drip (with Metro Boomin),https://www.last.fm/music/Offset/_/Ric+Flair+D...,None,travis scott,trap,hip-hop,rap,travis scott,trap,hip-hop,NaN
1090,Death Grips,Get Got,https://www.last.fm/music/Death+Grips/_/Get+Got,None,hip-hop,experimental hip-hop,rap,experimental,hip-hop,experimental,experimental hip-hop,NaN
1413,BTS,Epiphany,https://www.last.fm/music/BTS/_/Epiphany,None,pop,rnb,usa,pop,rnb,usa,NaN,NaN
1431,I DONT KNOW HOW BUT THEY FOUND ME,choke,https://www.last.fm/music/I+DONT+KNOW+HOW+BUT+...,None,hip-hop,experimental,noided,experimental hip-hop,industrial hip-hop,hip-hop,experimental,experimental hip-hop
1555,Frank Sinatra,Have Yourself A Merry Little Christmas - Remas...,https://www.last.fm/music/Frank+Sinatra/_/Have...,None,jazz,swing,oldies,vocal jazz,jazz,swing,christmas,NaN


# Пробую восстановить утерянные ссылки

In [85]:
search_query = "https://www.youtube.com/results?search_query="
yt_vid_link = "https://www.youtube.com/watch?v="
def get_yt_link(artist, name):
    link = search_query + artist + ' ' + name
    link = link.encode('ascii', 'ignore').decode('ascii')
    html = urllib.request.urlopen(link.replace(' ', '+'))
    video_ids = re.findall(r"watch\?v=(\S{11})", html.read().decode())
    return yt_vid_link + video_ids[0]

In [86]:
result_links = [get_yt_link(artist, name) for artist, name in zip(missing_yt['artist'], missing_yt['name'])]

In [106]:
df_w_genres.yt_links[df_w_genres.yt_links == "None"] = result_links

C:\Anaconda3\lib\site-packages\pandas\core\series.py:1020: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._where(~key, value, inplace=True)


In [107]:
df_w_genres.iloc[871]

artist                                                 Offset
name                       Ric Flair Drip (with Metro Boomin)
url         https://www.last.fm/music/Offset/_/Ric+Flair+D...
yt_links          https://www.youtube.com/watch?v=LPTlvQ1Zet0
Genre0                                           travis scott
Genre1                                                   trap
Genre2                                                hip-hop
Genre3                                                    rap
Genre4                                           travis scott
Genre5                                                   trap
Genre6                                                hip-hop
Genre7                                                    NaN
Name: 874, dtype: object

Worked flawless! :)